# AWS Resources deployment using a Jupyter Notebook

In [ ]:
!python3 -m pip install boto3
!python3 -m pip install requests
!python3 -m pip install tqdm
!python3 -m pip install pandas
!python3 -m pip install s3fs

In [ ]:
import IPython
import boto3
import time
import os
import json
import requests
import pandas as pd
from zipfile import ZipFile
from urllib.request import urlopen
from tqdm.notebook import tqdm

#### Some basic settings

In [74]:
my_bucket_name = 'dantohe-my-experimental-iac-01'
my_region = 'us-west-2'
stem = 'my-experimental'
my_InstanceProfileName = f'{stem}-InstanceProfileName-iac-02'
ec2_pem_name      = f'{stem}-kp-june-2021-01'
my_role_name = f'{stem}-ec2-role-02'

#### Create an EC2 key-pair    
If the key already exists then don't do anything.    

In [34]:
ec2 = boto3.client('ec2')
key_exists = False

response = ec2.describe_key_pairs()['KeyPairs']
for key in response:
    if key['KeyName'] == ec2_pem_name:
        key_exists = True
    found_instance = ec2.describe_instances(
        Filters=[
            {
                'Name': 'key-name',
                'Values': [key['KeyName']]
            }
        ]
    )['Reservations']
#     if len(found_instance) == 0:
#         print (key['KeyName'] + " is unused")

if key_exists:
    print('key already exists')
else:
    ec2_pem_path = f'./{ec2_pem_name}.pem'
    if os.path.isfile(ec2_pem_path):
        os.remove(ec2_pem_path)
    ec2_keypair = ec2.create_key_pair(KeyName=ec2_pem_name)
    with open(ec2_pem_path, 'w+') as ec2_pem_file:
        ec2_pem_file.write(str(ec2_keypair['KeyMaterial']))
    !chmod 400 {ec2_pem_path}
    print(f'{ec2_pem_name} has been created sucessfully and the pem is available at\n{ec2_pem_path}')
    

my-experimental-kp-june-2021-04 has been created sucessfully and the pem is available at
./my-experimental-kp-june-2021-04.pem


#### EC2 Resources    
IAM - refrences: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#role   


In [35]:
iam = boto3.client('iam')

In [70]:
roles = iam.list_roles()
role_list = roles['Roles']
existing_roles =[]
for key in role_list:
    existing_roles.append(key['RoleName'])
    #print(key['RoleName'])
    #print(key['Arn'])

if my_role_name in existing_roles:
    print(f'Role {my_role_name} already exists')
else:
    ec2_role = iam.create_role(
        Path='/',
        RoleName=my_role_name,
        Description='',
        MaxSessionDuration=3600,
        AssumeRolePolicyDocument="""{
      "Version": "2012-10-17",
      "Statement": [
        {
          "Effect": "Allow",
          "Principal": { "Service": "ec2.amazonaws.com"},
          "Action": "sts:AssumeRole"
        }
      ]
    }""".replace('<dw_bucket>', my_bucket_name))['Role']
    print(f'{my_role_name} has been createed')
#     ec2_role['Arn']

Role my-experimental-ec2-role-02 already exists


#### Attaches S3 acces policy to the role

In [60]:
for ec2_policy in  [
        'arn:aws:iam::aws:policy/AmazonS3FullAccess']:
    assert iam.attach_role_policy(
        RoleName=ec2_role['RoleName'],
        PolicyArn=ec2_policy)['ResponseMetadata']['HTTPStatusCode'] == 200

In [75]:
instance_profiles = iam.list_instance_profiles()
instance_profiles_list = instance_profiles['InstanceProfiles']
# print(instance_profiles_list)
existing_instance_profile_names =[]
for key in instance_profiles_list:
#     print(key['InstanceProfileName'])
    existing_instance_profile_names.append(key['InstanceProfileName'])
if my_InstanceProfileName in existing_instance_profile_names:
    print(f'{my_InstanceProfileName} already exists')
else:
    ec2_instance_profile = iam.create_instance_profile(InstanceProfileName=my_InstanceProfileName)['InstanceProfile']
    iam.get_waiter('instance_profile_exists').wait(InstanceProfileName=my_InstanceProfileName)
    print(f'{my_InstanceProfileName} has been created')
#     ec2_instance_profile['Arn']

my-experimental-InstanceProfileName-iac-02 has been created


#### Adding role to instance profile

In [90]:
assert iam.add_role_to_instance_profile(
    InstanceProfileName=ec2_instance_profile['InstanceProfileName'],
    RoleName=ec2_role['RoleName'])['ResponseMetadata']['HTTPStatusCode'] == 200

In [ ]:
ec2_sg = ec2.create_security_group(
    Description='Allows 22 trafic',
    GroupName='Airflow')
ec2.authorize_security_group_ingress(CidrIp='0.0.0.0/0', FromPort=22, ToPort=22, GroupId=ec2_sg['GroupId'], IpProtocol='TCP')
ec2.authorize_security_group_ingress(CidrIp='0.0.0.0/0', FromPort=8080, ToPort=8080, GroupId=ec2_sg['GroupId'], IpProtocol='TCP')
ec2.authorize_security_group_ingress(CidrIp='0.0.0.0/0', FromPort=5555, ToPort=5555, GroupId=ec2_sg['GroupId'], IpProtocol='TCP')
ec2.authorize_security_group_ingress(CidrIp='0.0.0.0/0', FromPort=3306, ToPort=3306, GroupId=ec2_sg['GroupId'], IpProtocol='TCP')
ec2_sg['GroupId']

## Clean Up

In [94]:
# my_InstanceProfileName = f'{stem}-InstanceProfileName-iac-02'
# ec2_pem_name      = f'{stem}-kp-june-2021-01'
# my_role_name = f'{stem}-ec2-role-02'

# removes role from instance profile
try:
    response = iam.remove_role_from_instance_profile(
        InstanceProfileName=my_InstanceProfileName,
        RoleName=my_role_name
    )
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f'The role {my_role_name} has been removed from {my_InstanceProfileName} \n\t{response}')
except Exception:
    print(f'The role {my_role_name} has been NOT BEEEN removed from {my_InstanceProfileName} - most likely it was not attached in the first place')
    pass  

# removes the instance profile 
try:
    response = iam.delete_instance_profile(InstanceProfileName=my_InstanceProfileName)
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f'The instance profile  {my_InstanceProfileName} has been removed \n\t{response}')
except Exception:
    print(f'The instance profile  {my_InstanceProfileName} has NOT BEEN removed most likely it did not exist in the first place')
    pass  


The role my-experimental-ec2-role-02 has been NOT BEEEN removed from my-experimental-InstanceProfileName-iac-02 - most likely it was not attached in the first place
The instance profile  my-experimental-InstanceProfileName-iac-02 has NOT BEEN removed most likely it did not exist in the first place
